In [13]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K

In [7]:
#data = pandas.read_csv("..")

In [2]:
# Fully Connected Network
T1 = Sequential()

T1.add(layers.Flatten(input_shape=(32,32)))
T1.add(layers.Dense(units=1024, activation="tanh"))
T1.add(layers.Dense(units=512, activation="sigmoid"))
T1.add(layers.Dense(units=100, activation="softmax"))

# Race classifier
T1.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T1.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
#T1.add(layers.Dense(units=10, activation="softmax"))

print(T1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1049600   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               51300     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 1,626,104
Trainable params: 1,626,104
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
# CNN
T2 = Sequential()

T2.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation="relu", strides=(1,1), padding="same", input_shape=(32,32,1)))
T2.add(layers.MaxPool2D(pool_size=(2,2)))
T2.add(layers.Flatten())
T2.add(layers.Dense(units=100, activation="relu"))

# Race classifier
#T2.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T2.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
T2.add(layers.Dense(units=10, activation="softmax"))

print(T2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 40)        1040      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 40)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10240)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1024100   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 1,026,150
Trainable params: 1,026,150
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# Custom CNN - LeNet-5

T3 = Sequential()

# NEED TO CHOOSE PARAMS
T3.add(layers.Conv2D(filters=6, kernel_size=(5,5), activation="sigmoid", input_shape=(32,32,1)))
T3.add(layers.AveragePooling2D(pool_size=(2,2)))
T3.add(layers.Conv2D(filters=16, kernel_size=(5,5), activation="sigmoid"))
T3.add(layers.AveragePooling2D(pool_size=(2,2)))
T3.add(layers.Flatten())
T3.add(layers.Dense(units=120, activation="sigmoid"))
T3.add(layers.Dense(units=84, activation="sigmoid"))
T3.add(layers.Dense(units=10, activation="sigmoid"))

# Race classifier
#T3.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T3.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
T3.add(layers.Dense(units=10, activation="softmax"))

print(T3.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_4 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_5 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_8 (Dense)              (None, 84)               

In [5]:
# Parallel CNN
T4 = Sequential()

In [19]:
# VAE
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(z_log_var) * epsilon

latent_dim = 8 # NEEDS TO BE AT LEAST 5
inputs = keras.Input(shape=(32,32,1))
# FIX ENCODER LAYER PARAMS
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)
z = layers.Lambda(sampling)([z_mean, z_log_var])

encoder = keras.models.Model(inputs, [z_mean, z_log_var, z])
print(encoder.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 16, 16, 32)   320         input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 8, 8, 64)     18496       conv2d_24[0][0]                  
__________________________________________________________________________________________________
dense_21 (Dense)                (None, 8, 8, 8)      520         conv2d_25[0][0]                  
_______________________________________________________________________________________

In [23]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8*8*64, activation="relu")(latent_inputs)
x = layers.Reshape((8,8,64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)

decoder = keras.models.Model(latent_inputs, decoder_outputs)
print(decoder.summary())

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_25 (Dense)             (None, 4096)              36864     
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 32)        18464     
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 32, 32, 1)         289       
Total params: 92,545
Trainable params: 92,545
Non-trainable params: 0
__________________________________________________